In [ ]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR, CosineAnnealingWarmRestarts
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds
from tools import EarlyStopping

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
train_folder = 'data/train/'
test_folder = 'data/test/'
train_label_path = 'data/train_label.csv'

In [ ]:
train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

In [ ]:
num_class = len(train_label['label'].unique())

In [ ]:
# 모든 csv 파일의 상태_B로 변화는 시점이 같다라고 가정
# 하지만, 개별 csv파일의 상태_B로 변화는 시점은 상이할 수 있음
def data_loader_all_v2(func, files, folder='', train_label=None, event_time=10, nrows=60):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)     
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close()
        pool.join()        
    combined_df = pd.concat(df_list)    
    return combined_df

In [ ]:
# train = data_loader_all_v2(data_loader_v2, train_list, folder=train_folder, train_label=train_label, 
#                            event_time=10, nrows=120)


In [ ]:
train = joblib.load('data/df_train_10_200.pkl').reset_index()
test = joblib.load('data/df_test_10.pkl')


In [ ]:
y_cols = 'label'
fea_cols = [c for c in train.columns if c[0] == 'V']
len(fea_cols), y_cols

In [ ]:
scaler = joblib.load('scaler_20200129T135731.bin')

train[fea_cols] = scaler.transform(train[fea_cols].values)
test[fea_cols] = scaler.transform(test[fea_cols].values)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=81511991154 % 2**32-1)

# X_train.shape, X_test.shape

In [ ]:
class Dataset15(Dataset):
    def __init__(self, df, fea_cols, y_cols):        
        self.X = df[fea_cols].values
#         self.y = pd.get_dummies(df[y_cols]).values
        self.y = df[y_cols].values

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32), self.y[idx].astype(np.long)
    

In [ ]:
class Trainer(object):
    def __init__(self, model, criterion, optimizer, scheduler, device):
        self.device = device
        self.model = model#.to(self.device)
        self.criterion = criterion#.to(self.device)
        self.optimizer = optimizer
        self.scheduler = scheduler

        print(self.model.train())
        pass
    
    def train(self, data_loader, epoch):
        self.model.train()
        total_loss = 0
        
        for i, data in enumerate(data_loader):
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            
            self.scheduler.step(epoch + i / len(data_loader))
            self.optimizer.zero_grad()
            y_pred = self.model(X_batch)
#             print(y_pred, y_batch)
            
            loss = self.criterion(y_pred, y_batch)
            total_loss = total_loss + loss.item()
            
            loss.backward()
            self.optimizer.step()
        
        
        return total_loss / len(data_loader)
    
    def eval(self, data_loader):
        self.model.eval()
        total_loss = 0
#         print('valid_loader', len(valid_loader))
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            with torch.no_grad():
                y_pred = self.model(X_batch)
                loss = self.criterion(y_pred, y_batch)
                total_loss = total_loss + loss.item()
        return total_loss / len(data_loader)

    def save(self, model_path='checkpoint.pt'):
#         torch.save(self.model.state_dict(), 'checkpoint.pt')
        joblib.dump(self.model, model_path)
        return
    
    def load(self, model_path='checkpoint.pt'):
#         self.model.load_state_dict(torch.load(model_path))
        self.model = joblib.load(model_path)
        return

#### Model

In [ ]:
class CNNModel(torch.nn.Module):
    def __init__(self, dropout_probability=0.5):
        super().__init__()
        relu = torch.nn.ReLU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.cnn = torch.nn.Sequential(
            torch.nn.Conv1d(1, 2, 3, stride=1, padding=1), torch.nn.BatchNorm1d(2), relu,
            torch.nn.Conv1d(2, 4, 3, stride=1, padding=1), torch.nn.BatchNorm1d(4), relu,
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(4, 4, 3, stride=1, padding=1), torch.nn.BatchNorm1d(4), relu,
            torch.nn.Conv1d(4, 8, 3, stride=1, padding=1), torch.nn.BatchNorm1d(8), relu,
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(8, 8, 3, stride=1, padding=1), torch.nn.BatchNorm1d(8), relu,
            torch.nn.Conv1d(8, 8, 3, stride=1, padding=1), torch.nn.BatchNorm1d(8), relu,
            torch.nn.MaxPool1d(2),
        )
            
        self.clf = torch.nn.Sequential(
            torch.nn.Linear(5120, 3000), torch.nn.BatchNorm1d(3000), relu, dropout,
            torch.nn.Linear(3000, 1024), torch.nn.BatchNorm1d(1024), relu, dropout,
            torch.nn.Linear(1024, 198)
        )
        
    def forward(self, x):
        x = x.unsqueeze(1)
        out = self.cnn(x)
        dim = 1
        for d in out.size()[1:]: #24, 4, 4
            dim = dim * d
        out = out.view(-1, dim)
        out = self.clf(out)
        return out



#### Train

In [ ]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
print(model_ts)

# print(f'fea_size {len(fea_cols)} layer_cols {layer_cols}')

torch.manual_seed(81511991154)
torch.initial_seed()

In [ ]:
dataset = Dataset15(train[fea_cols + [y_cols]], fea_cols, y_cols)

train_set, val_set = torch.utils.data.random_split(dataset, [int(train.shape[0] * 0.8), int(train.shape[0] * 0.2)])

print(len(train_set), len(val_set))

batch_size = 3000
num_workers = 8

all_loader = DataLoader(dataset=dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=len(val_set))

print(f'batch_size {batch_size} num_workers {num_workers}')
print(f'train_loader {len(train_loader)} val_loader {len(val_loader)}')

##### Trainer

In [ ]:
total_epoch = 10000
patience = 200
val_loss_min = np.Inf

early_stopping = EarlyStopping(patience=patience, min_epoch=1, verbose=True)
early_stopping.val_loss_min = val_loss_min
early_stopping.best_score = None if val_loss_min==np.Inf else -val_loss_min 

model = CNNModel(dropout_probability=0).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

scheduler = StepLR(optimizer, step_size=100, gamma=0.5)
# scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=100)

criterion = nn.CrossEntropyLoss(reduction='mean').to(device)

trainer = Trainer(model, criterion, optimizer, scheduler, device)


for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
    if os.path.isfile('stop.flag'):
        print(f'{e} stop!')
        break

    train_loss = trainer.train(all_loader, e)

    if e % 1 == 0:
#         valid_loss = trainer.eval(val_loader)
        valid_loss = train_loss

        ts = datetime.now().strftime('%Y%m%dT%H%M%S')
        print(f'[{ts}] Epock {e} / {total_epoch}\t lr {trainer.scheduler.get_lr()[0]}')
        print(f'  train_loss: {train_loss}  valid_loss: {valid_loss}')

        early_stopping(valid_loss, model)

        if early_stopping.early_stop:
            print("\tEarly stopping epoch {}, valid loss {}".format(e, early_stopping.val_loss_min))
            break


model.load_state_dict(torch.load('model/checkpoint.pt'))
val_loss_min = early_stopping.val_loss_min

model_path = 'model/{}_{}'.format(model_ts, val_loss_min)
trainer.save('{}.model'.format(model_path))
print(model_path)


In [ ]:
print(test.shape)

model.eval()
model = model.cpu()
y_pred = model(torch.Tensor(test[fea_cols].values))    

y_pred = F.softmax(y_pred)

print(y_pred)

In [ ]:
model_tag ='{}cnn_{}_{}'.format(model_ts, train_loss, valid_loss)
# model_tag ='{}'.format(model_ts)

In [ ]:
submission = pd.DataFrame(data=y_pred.cpu().detach().numpy())
submission.index = test.index
submission.index.name = 'id'
submission = submission.sort_index()
submission = submission.groupby('id').mean()

submission.to_csv('submit/{}.csv'.format(model_tag), index=True) 
model_tag

In [ ]:
submission